It is highly recommended to use a powerful **GPU**, you can use it for free uploading this notebook to [Google Colab](https://colab.research.google.com/notebooks/intro.ipynb).
<table align="center">
 <td align="center"><a target="_blank" href="https://colab.research.google.com/github/ezponda/intro_deep_learning/blob/main/class/Fundamentals/First_Model.ipynb">
        <img src="https://i.ibb.co/2P3SLwK/colab.png"  style="padding-bottom:5px;" />Run in Google Colab</a></td>
  <td align="center"><a target="_blank" href="https://github.com/ezponda/intro_deep_learning/blob/main/class/Fundamentals/First_Model.ipynb">
        <img src="https://i.ibb.co/xfJbPmL/github.png"  height="70px" style="padding-bottom:5px;"  />View Source on GitHub</a></td>
</table>

## Load and preprocess dataset

In [1]:
from tarfile import data_filter

import pandas as pd
import matplotlib.pyplot as plt

from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# 1. Data
X, y = make_moons(n_samples=1000, noise=0.2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

## Pytorch Model

In [12]:
# 2. Model
class MLP(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(2, 16),
            nn.ReLU(),
            nn.Linear(16, 2)
        )

    def forward(self, x):
        return self.net(x)


In [13]:
model = MLP()

In [14]:
model

MLP(
  (net): Sequential(
    (0): Linear(in_features=2, out_features=16, bias=True)
    (1): ReLU()
    (2): Linear(in_features=16, out_features=2, bias=True)
  )
)

## Training

In [15]:
# 3. Training setup
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

epochs = 10
print_every = 1  # Print loss every 'print_every' epochs

In [21]:
# 4. Training loop
model.train()

losses = []

for epoch in range(epochs):
    
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    
    loss = loss_fn(outputs, y_train_tensor)
    losses.append(loss.item())
    
    loss.backward()
    optimizer.step()
    
    if epoch % print_every == 0:
        acc = (outputs.argmax(1) == y_train_tensor).float().mean()
        print(f"Epoch {epoch} | Loss: {loss.item():.4f} | Train Acc: {acc:.4f}")

Epoch 0 | Loss: 0.3842 | Train Acc: 0.8462
Epoch 1 | Loss: 0.3682 | Train Acc: 0.8487
Epoch 2 | Loss: 0.3541 | Train Acc: 0.8475
Epoch 3 | Loss: 0.3419 | Train Acc: 0.8462
Epoch 4 | Loss: 0.3314 | Train Acc: 0.8475
Epoch 5 | Loss: 0.3225 | Train Acc: 0.8475
Epoch 6 | Loss: 0.3149 | Train Acc: 0.8512
Epoch 7 | Loss: 0.3087 | Train Acc: 0.8512
Epoch 8 | Loss: 0.3035 | Train Acc: 0.8575
Epoch 9 | Loss: 0.2992 | Train Acc: 0.8575


## Evaluation

In [22]:
# 5. Evaluation
model.eval()

MLP(
  (net): Sequential(
    (0): Linear(in_features=2, out_features=16, bias=True)
    (1): ReLU()
    (2): Linear(in_features=16, out_features=2, bias=True)
  )
)

In [23]:
with torch.no_grad():
    test_outputs = model(X_test_tensor)
    test_preds = test_outputs.argmax(1)
    test_acc = (test_preds == y_test_tensor).float().mean()
    print(f"\nTest Accuracy: {test_acc:.4f}")


Test Accuracy: 0.8400


In [24]:
hist = {
    'epoch': list(range(epochs)),
    'loss': losses,
}

In [25]:
def show_loss_accuracy_evolution(hist):

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Sparse Categorical Crossentropy')
    ax1.plot(hist['epoch'], hist['loss'], label='Train Error')
    ax1.grid()
    ax1.legend()


## Practice.

- Create a function that receives the number of neurons in the hidden layer and returns a model.

- Create a function that computes the loss and accuracy of the model in a given subset of the dataset.

- Change the number of neurons in the hidden layer to 32 and 64. What happens with the accuracy ?

- Split the dataset in training and validation and plot the accuracy of both datasets

- Match the same plot as with the Keras implementation using PyTorch. 



In [29]:
# Create models with different number of neurons
def create_model(n_neurons):
    model = nn.Sequential(
        nn.Linear(2, n_neurons),
        nn.ReLU(),
        nn.Linear(n_neurons, 2)
    )
    return model

In [30]:
# Compute loss and accuracy on a given dataset
def compute_loss_and_accuracy(model, X_tensor, y_tensor):
    with torch.no_grad():
        outputs = model(X_tensor)
        loss = loss_fn(outputs, y_tensor)
        acc = (outputs.argmax(1) == y_tensor).float().mean()
    return loss.item(), acc.item()

In [ ]:
# Split the dataset into training and validation